In [34]:
%matplotlib qt
import os
import pickle
from sdypy import EMA
import numpy as np
import matplotlib.pyplot as plt 
from EMA_functions import *
import pandas as pd
import matplotlib.animation as animation
import matplotlib.cm as cm
import matplotlib.colors as col
import ast
from scipy.io import loadmat
from pixel_setter import play_video

root_drive_sim = os.path.normpath('G:/.shortcut-targets-by-id/1k1B8zPb3T8H7y6x0irFZnzzmfQPHMRPx/Illimited Lab Projects/Research Projects/Spiders/Simulations')
root_EMA = 'C:/Users/thijsmas/Documents/GitHub/pyidi_data/EMA structure'

Input 2D case

In [35]:
file_name = 'measurement_0_00_degres_S01'

roi_size = (9,9)
method = '2d'
file_name_EMA = file_name + '_' + method + f'_roi{roi_size[0]}x{roi_size[1]}.pkl'
with open(os.path.join(root_EMA, file_name_EMA), 'rb') as f:
    EMA_structure2D = pkl.load(f)
video = EMA_structure2D.open_video(add_extension=False)

EMA_structure2D.tp = EMA_structure2D.maxima
EMA_structure2D.d  = EMA_structure2D.displacements
td_2d = EMA_structure2D.d +  EMA_structure2D.maxima.reshape(len(EMA_structure2D.maxima),1,2)
dt = 1/video.info['Record Rate(fps)']
t_vec = np.arange(0, video.N)*dt

D:/thijsmas/HSC - Ladisk\transfer_100504_files_37b9d3ac\measurement_0_00_degres_S01.cihx


Input 1D case

In [36]:
EMA_structure1D_vec = []
roi_size = (9,9)
method = '1d'

theta_vec = np.linspace(0, np.pi, 5)
theta_vec = theta_vec[:-1]
theta_vec = np.append(theta_vec, 1.5730517285207863)
for theta in theta_vec:
    s = np.array([np.cos(theta), -np.sin(theta)])
    s = np.round(s, 2)
    file_name_EMA = file_name + '_' + method + f'_roi{roi_size[0]}x{roi_size[1]}_theta{theta}.pkl'
    with open(os.path.join(root_EMA, file_name_EMA), 'rb') as f:
        EMA_structure = pkl.load(f)
    EMA_structure.theta = theta
    EMA_structure.s = s
    EMA_structure.tp = EMA_structure.maxima
    EMA_structure.d  = EMA_structure.displacements
    EMA_structure.td = EMA_structure.d +  EMA_structure.maxima.reshape(len(EMA_structure.maxima),1,2)
    EMA_structure1D_vec.append(EMA_structure)

tp_stack = np.stack([EMA_structure1D.tp for EMA_structure1D in EMA_structure1D_vec], axis=0)
tp_stack = np.vstack((EMA_structure2D.tp[np.newaxis], tp_stack))
# file_name_EMA = file_name + '_' + method + f'_roi{roi_size[0]}x{roi_size[1]}_theta{theta}.pkl'
# with open(os.path.join(root_EMA, file_name_EMA), 'rb') as f:
#     EMA_structure = pkl.load(f)

# EMA_structure.tp = EMA_structure.maxima
# EMA_structure.d  = EMA_structure.displacements
# td = EMA_structure.d +  EMA_structure.maxima.reshape(len(EMA_structure.maxima),1,2)
tp_stack.shape, EMA_structure.tp.shape

((6, 120, 2), (120, 2))

In [37]:
%matplotlib qt
sequential_image_n = 0
# EMA_structure.nut_idx((1012, 271), exclude_high_amplitude = True, d_lim =10)
EMA_structure1D_vec[0].nut_idx((1012, 271), exclude_high_amplitude = True, d_lim =10)
# fig, ax = EMA_structure.plot_still_frame(video, sequential_image_n, show_saturation = False, tp_nut = True, tp = True, valid_only=False, bit_depth = 16)
fig, ax = EMA_structure1D_vec[0].plot_still_frame(video, sequential_image_n, show_saturation = False, tp_nut = True, tp = tp_stack[:-1], valid_only=False, bit_depth = 16, labels = ['2D', '1D - [1. 0.]', '1D - [0.7071 -0.7071]', '1D - [0. 1.]', '1D - [-0.7071 -0.7071]', '1D - [0.9995, -0.0315]'])
fig.set_tight_layout(True)
ax.set_xticks([])
ax.set_yticks([])
n_path_frames = 100
d_scaler =30
for EMA_structure in EMA_structure1D_vec[:-1]:
    td = d_scaler*EMA_structure.d +  EMA_structure.maxima.reshape(len(EMA_structure.maxima),1,2)
    for td_p in td:
        ax.plot(td_p[:n_path_frames,1], td_p[:n_path_frames,0], 'k-', alpha = 0.5)
        ax.plot(td_p[:n_path_frames,1], td_p[:n_path_frames,0], 'w--', alpha = 0.5)
td = d_scaler*EMA_structure2D.d +  EMA_structure2D.maxima.reshape(len(EMA_structure2D.maxima),1,2)
for td_p in td:
    ax.plot(td_p[:n_path_frames,1], td_p[:n_path_frames,0], 'k-', alpha = 0.5)
    ax.plot(td_p[:n_path_frames,1], td_p[:n_path_frames,0], 'w--', alpha = 0.5)
    



C:\Users\thijsmas\AppData\Local\Temp\ipykernel_26788\2347472301.py:7: PendingDeprecationWarning: The set_tight_layout function will be deprecated in a future version. Use set_layout_engine instead.
  fig.set_tight_layout(True)


In [38]:
%matplotlib qt
td = EMA_structure1D_vec[1].d +  EMA_structure1D_vec[1].maxima.reshape(len(EMA_structure1D_vec[1].maxima),1,2)
first_frame = 1
ani = play_video(video, range(1000, video.N-1), interval=30, points=td, axis=([0, video.info['Image Width']],[video.info['Image Height'], 0]))

In [39]:
n = EMA_structure2D.d.shape[1]
disp_fft_2D_i = np.fft.rfft(EMA_structure2D.d[:,:,0], n=n, axis=1) / n
disp_fft_2D_j = np.fft.rfft(EMA_structure2D.d[:,:,1], n=n, axis=1) / n

# As we have effectively discarded the negative frequency spectra, the positive frequency components are doubled
disp_fft_2D_i[1:] *= 2
disp_fft_2D_j[1:] *= 2

disp_fft_2D = disp_fft_2D_i * s[1] + disp_fft_2D_j * s[0]
disp_fft_2D_stack = np.stack((disp_fft_2D_i, disp_fft_2D_j), axis=2)
disp_fft_2D_norm = np.linalg.norm(disp_fft_2D_stack, axis=2)

freq = np.fft.rfftfreq(n, dt)

T = t_vec[-1]
S_2D = 1/T * np.conj(disp_fft_2D) * disp_fft_2D
S_2D_mean = np.mean(S_2D, axis=0)

S_2D_i = 1/T * np.conj(disp_fft_2D_i) * disp_fft_2D_i
S_2D_i_mean = np.mean(S_2D_i, axis=0)

S_2D_j = 1/T * np.conj(disp_fft_2D_j) * disp_fft_2D_j
S_2D_j_mean = np.mean(S_2D_j, axis=0)

S_2D_norm = 1/T * np.conj(disp_fft_2D_norm) * disp_fft_2D_norm
S_2D_norm_mean = np.mean(S_2D_norm, axis=0)

In [40]:
for EMA_structure in EMA_structure1D_vec:
    EMA_structure.d = EMA_structure.d - EMA_structure2D.d
    EMA_structure.disp_fft_i = np.fft.rfft(EMA_structure.d[:,:,0], n=n, axis=1) / n
    EMA_structure.disp_fft_j = np.fft.rfft(EMA_structure.d[:,:,1], n=n, axis=1) / n

    # As we have effectively discarded the negative frequency spectra, the positive frequency components are doubled
    EMA_structure.disp_fft_i[1:] *= 2
    EMA_structure.disp_fft_j[1:] *= 2

    EMA_structure.disp_fft = EMA_structure.disp_fft_i * EMA_structure.s[1] + EMA_structure.disp_fft_j * EMA_structure.s[0]

    EMA_structure.freq = np.fft.rfftfreq(n, dt)

    EMA_structure.S_1D = 1/T * np.conj(EMA_structure.disp_fft) * EMA_structure.disp_fft
    EMA_structure.S_1D_mean = np.mean(EMA_structure.S_1D, axis=0)
    
    # mean_angle, s_mean

In [41]:
angle_vec = np.arctan2(EMA_structure2D.d[:,:,0], EMA_structure2D.d[:,:,1])
mean_angle = np.mean(angle_vec)
s_mean = np.array([np.cos(mean_angle), np.sin(mean_angle)])
s_mean

array([0.99950332, 0.03151362])

In [42]:
%matplotlib qt
fig, ax = plt.subplots(len(EMA_structure1D_vec)-1,1, figsize=(10,5))
for i, EMA_structure in enumerate(EMA_structure1D_vec[:-1]):
    ax[i].plot(freq, EMA_structure.S_1D_mean, 'k', label='Mean 1D')
    ax[i].plot(freq, S_2D_mean, 'r:', label='2D - projection')
    ax[i].plot(freq, S_2D_norm_mean, 'b:', label='Mean 2D norm')
    ax[i].set_yscale('log')
    ax[i].set_yscale('log')
    ax[i].legend()
    ax[i].set_title(f's = {EMA_structure.s}')
    
    ax[i].set_ylabel('Auto-spectral density')
    ax[i].set_xlim([450, 1000])
    ax[i].set_ylim([1e-10, 1e-5])
ax[-1].set_xlabel('Frequency [Hz]')

Text(0.5, 0, 'Frequency [Hz]')

In [43]:
t_interest = 0.02
t_i = np.abs(t_vec - t_interest).argmin()
fig, ax = plt.subplots(1, 1, figsize=(15, 5))
for disp_ij in EMA_structure.d:
    ax.plot(t_vec[:t_i], disp_ij[:t_i,0], 'r', alpha=0.2, linewidth=0.2)
    ax.plot(t_vec[:t_i], disp_ij[:t_i,1], 'b', alpha=0.2, linewidth=0.2)
    # ax.plot(EMA_structure.t_camera_raw, disp_i, 'r', alpha=0.2, linewidth=0.2)